In [1]:
if not os.path.isfile("estrutura_ud.py"):
    ! wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/estrutura_ud.py
import estrutura_ud

--2021-03-20 00:26:56--  https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/estrutura_ud.py
Resolvendo raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Conectando-se a raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 7332 (7,2K) [text/plain]
Salvando em: “estrutura_ud.py”

estrutura_ud.py     100%[===================>]   7,16K  --.-KB/s    em 0,09s   

2021-03-20 00:26:57 (81,4 KB/s) - “estrutura_ud.py” salvo [7332/7332]



In [15]:
corpus = estrutura_ud.Corpus()
corpus.load("../Bosque/bosque-workbench.conllu")

for i in range(2):
    for sentence in corpus.sentences.values():
        if sentence.sent_id not in [
            "1-20140905-TESEDSC_0_resumo-11",
            "32-20141110-MONOGRAFIA_0_resumo-19",
            "36-20140327-TESEMSC_0_resumo-6",
            "51-20140924-MONOGRAFIA_0_resumo-1",
            "6-20140908-MONOGRAFIA_0_intro-53",
            '105-20151215-MONOGRAFIA_0_resumo-8',
            "32-20141110-MONOGRAFIA_0_resumo-13",
            "20-20140904-TESEDSC_0-59",
            "20-20140904-TESEDSC_0-66",
            "19-20140916-TESEDSC_0_resumo-9"
        ]:
            tokens = sentence.tokens
            for token in tokens:
                if not '-' in token.id and token.deprel not in ["appos:parataxis", "compound", "flat:foreign", "orphan"]:
                    has_cc = False
                    has_colon = False
                    has_comma = False
                    has_parenthesis = False
                    for _token in tokens:
                        if _token.dephead == token.id:
                            if _token.deprel == "cc":
                                has_cc = True
                            if _token.word in [",", "-", "–", ";"]:
                                has_comma = True
                            if _token.word == "(":
                                has_parenthesis = True
                        if _token.dephead == token.id or _token.dephead == token.head_token.id:
                            if _token.word == ":":
                                has_colon = True

                    if token.upos == "NOUN" and token.head_token.upos == "NOUN" and int(token.id) > int(token.head_token.id):
                        if not any([has_comma, has_cc, has_colon, has_parenthesis]):
                            token.deprel = "nmod"

                    if token.upos == "NUM" and token.head_token.upos == "NOUN" and int(token.id) > int(token.head_token.id):
                        if not any([has_comma, has_cc, has_colon, has_parenthesis]):
                            token.deprel = "nummod"

                    if token.upos == "PROPN" and token.head_token.upos == "NOUN" and int(token.id) > int(token.head_token.id):
                        if not any([has_colon, has_comma, has_parenthesis, has_cc]):
                            token.deprel = "nmod"

                    if token.deprel == "appos:parataxis":
                        token.deprel = "nmod:appos"
                    
                    if token.upos == "PROPN" and token.head_token.upos == "PROPN" and int(token.id) > int(token.head_token.id):
                        if not any([has_cc, has_comma, has_parenthesis]):
                            token.deprel = "flat:name"
                            if token.head_token == "flat:name":
                                token.dephead = token.head_token.dephead
                            for _token in tokens:
                                if _token.dephead == token.id:
                                    if _token.upos in ["ADP", "DET"]:
                                        _token.dephead = token.dephead
                                        _token.deprel = "flat:name"

                    if token.head_token.deprel == "flat:name":
                        token.dephead = token.head_token.dephead# if token.head_token.head_token.deprel != "flat:name" else token.head_token.head_token.dephead
                        #if not any([has_cc, has_comma]):
                        token.deprel = "flat:name"
                    

corpus.save("teste.conllu")
! meld --diff Petroles.conllu teste.conllu
os.remove("teste.conllu")

build: 0.8158910274505615